In [24]:
# Importing required libraries
import pandas as pd

# Loading the first file
file_path_1 = '2Apurm.csv'
file_1 = pd.read_csv(file_path_1)

# Loading the second file
file_path_2 = '2Apurp.csv'
file_2 = pd.read_csv(file_path_2)

In [25]:
# Correcting the 'Date' column name by removing the extra space
file_1.rename(columns={' Date': 'Date'}, inplace=True)
file_2.rename(columns={' Date': 'Date'}, inplace=True)

# Converting the date columns to datetime objects for accurate comparison
file_1['Date'] = pd.to_datetime(file_1['Date'], dayfirst=True)
file_2['Date'] = pd.to_datetime(file_2['Date'], dayfirst=True)

In [26]:
file_1.head()

,GST,Trade Name,Doc Number,Date,Taxable Value,IGST,CGST,SGST,REM
0,03AAECJ9456C1ZP,JOLLY HEALTHCARE PVT LTD OCT 2020,519,2022-04-29,3996.48,199.82,0.0,0.0,MARG
1,03AAECJ9456C1ZP,JOLLY HEALTHCARE PVT LTD OCT 2020,519,2022-04-29,36220.80,4346.50,0.0,0.0,MARG
2,03AAECJ9456C1ZP,JOLLY HEALTHCARE PVT LTD OCT 2020,519,2022-04-29,88600.07,10632.01,0.0,0.0,MARG
3,03AAECJ9456C1ZP,JOLLY HEALTHCARE PVT LTD OCT 2020,538,2022-05-31,31693.20,3803.18,0.0,0.0,MARG
4,03AAECJ9456C1ZP,JOLLY HEALTHCARE PVT LTD OCT 2020,538,2022-05-31,83058.26,9966.99,0.0,0.0,MARG


In [27]:
file_2.head()

,GST,Trade Name,Doc Number,Date,Taxable Value,IGST,CGST,SGST,REM
0,03AAECJ9456C1ZP,JOLLY HEALTH CARE PRIVATE LIMITED,JHC/519/2022-23,2022-04-29,124820.87,14978.51,0.0,0.0,PORTAL
1,03AAECJ9456C1ZP,JOLLY HEALTH CARE PRIVATE LIMITED,JHC/519/2022-23,2022-04-29,3996.48,199.82,0.0,0.0,PORTAL
2,03AAECJ9456C1ZP,JOLLY HEALTH CARE PRIVATE LIMITED,JHC/538/2022-23,2022-05-31,114751.46,13770.17,0.0,0.0,PORTAL
3,03AAECJ9456C1ZP,JOLLY HEALTH CARE PRIVATE LIMITED,JHC/556/2022-23,2022-06-29,111552.16,13386.25,0.0,0.0,PORTAL
4,03AAECJ9456C1ZP,JOLLY HEALTH CARE PRIVATE LIMITED,JHC/556/2022-23,2022-06-29,3691.38,184.57,0.0,0.0,PORTAL


In [28]:
# Function to find the closest match based on a weighted combination of date and taxable value
def closest_match(target_date, target_value, date_list, value_list, date_weight=0.05, value_weight=1):
    min_combined_score = float('inf')
    closest_idx = None
    for idx, (date, value) in enumerate(zip(date_list, value_list)):
        if date <= target_date:
            date_difference = abs(target_date - date).days * date_weight
            value_difference = abs(target_value - value) / target_value * 100 * value_weight
            combined_score = date_difference + value_difference
            if combined_score < min_combined_score:
                min_combined_score = combined_score
                closest_idx = idx
    return closest_idx

In [29]:
# Repeating the analysis as described earlier

# Lists to store the results
exact_matches = []
under_5_percent_diff = []
under_10_percent_diff = []
more_than_10_percent_diff = []
not_found_in_portal = []
not_found_in_marg = []

In [30]:
# Iterating through the bills in file 1 (MARG)
for idx, row in file_1.iterrows():
    gst = row['GST']
    date = row['Date']
    taxable_value = row['Taxable Value']
    
    # Finding bills with the same GST number in file 2 (Portal)
    matching_gst_rows = file_2[file_2['GST'] == gst]
    
    if not matching_gst_rows.empty:
        # Finding the closest match based on date and taxable value
        closest_idx = closest_match(date, taxable_value, matching_gst_rows['Date'], matching_gst_rows['Taxable Value'])
        
        # If a closest match is found
        if closest_idx is not None:
            closest_row = matching_gst_rows.iloc[closest_idx]

            # Calculating the percentage difference in taxable value
            value_diff_percent = abs(taxable_value - closest_row['Taxable Value']) / taxable_value * 100

            # Categorizing based on the percentage difference
            if value_diff_percent == 0:
                exact_matches.append((row, closest_row))
            elif value_diff_percent < 5:
                under_5_percent_diff.append((row, closest_row))
            elif value_diff_percent < 10:
                under_10_percent_diff.append((row, closest_row))
            else:
                more_than_10_percent_diff.append((row, closest_row))

            # Removing the matched row from file 2
            file_2.drop(closest_row.name, inplace=True)
        else:
            # Not found in Portal file (no close taxable value found)
            not_found_in_portal.append(row)

# Bills remaining in file 2 are not found in MARG file
not_found_in_marg = file_2

In [31]:
# Converting results to DataFrames for better visualization
exact_matches_df = pd.DataFrame([item[0] for item in exact_matches])
under_5_percent_diff_df = pd.DataFrame([item[0] for item in under_5_percent_diff])
under_10_percent_diff_df = pd.DataFrame([item[0] for item in under_10_percent_diff])
more_than_10_percent_diff_df = pd.DataFrame([item[0] for item in more_than_10_percent_diff])
not_found_in_portal_df = pd.DataFrame(not_found_in_portal)
not_found_in_marg_df = pd.DataFrame(not_found_in_marg)

In [32]:
# Summary of results
result_summary = {
    "Exact Matches": len(exact_matches_df),
    "Under 5% Difference": len(under_5_percent_diff_df),
    "Under 10% Difference": len(under_10_percent_diff_df),
    "More Than 10% Difference": len(more_than_10_percent_diff_df),
    "Not Found in Portal File": len(not_found_in_portal_df),
    "Not Found in MARG File": len(not_found_in_marg_df),
}

result_summary

{'Exact Matches': 1786,
 'Under 5% Difference': 794,
 'Under 10% Difference': 43,
 'More Than 10% Difference': 522,
 'Not Found in Portal File': 445,
 'Not Found in MARG File': 142}

In [33]:
# Summary of results
result_summary = {
    "Exact Matches": len(exact_matches_df),
    "Under 5% Difference": len(under_5_percent_diff_df),
    "Under 10% Difference": len(under_10_percent_diff_df),
    "More Than 10% Difference": len(more_than_10_percent_diff_df),
    "Not Found in Portal File": len(not_found_in_portal_df),
    "Not Found in MARG File": len(not_found_in_marg_df),
}

result_summary

{'Exact Matches': 1786,
 'Under 5% Difference': 794,
 'Under 10% Difference': 43,
 'More Than 10% Difference': 522,
 'Not Found in Portal File': 445,
 'Not Found in MARG File': 142}

In [34]:
exact_matches_df

,GST,Trade Name,Doc Number,Date,Taxable Value,IGST,CGST,SGST,REM
0,03AAECJ9456C1ZP,JOLLY HEALTHCARE PVT LTD OCT 2020,519,2022-04-29,3996.48,199.82,0.0,0.0,MARG
8,03AAECJ9456C1ZP,JOLLY HEALTHCARE PVT LTD OCT 2020,593,2022-08-25,1845.69,92.28,0.0,0.0,MARG
11,03AAECJ9456C1ZP,JOLLY HEALTHCARE PVT LTD OCT 2020,604,2022-09-09,164401.92,19728.23,0.0,0.0,MARG
12,03AAECJ9456C1ZP,JOLLY HEALTHCARE PVT LTD OCT 2020,609,2022-09-14,164401.92,19728.23,0.0,0.0,MARG
13,03AAECJ9456C1ZP,JOLLY HEALTHCARE PVT LTD OCT 2020,634,2022-10-21,1845.69,92.28,0.0,0.0,MARG
...,...,...,...,...,...,...,...,...,...
3601,27AABPJ9899D1ZA,MAHAVIR AGENCY BORIWALI MUMBAI,4226,2022-10-03,37190.47,1115.71,0.0,0.0,MARG
3602,27AACCH2305Q1ZG,HEALTH N U THERAPEUTICS PVT LTD,2300155,2022-04-28,12756.80,1530.81,0.0,0.0,MARG
3611,27AACCH2305Q1ZG,HEALTH N U THERAPEUTICS PVT LTD,2301729,2022-10-29,16996.20,2039.54,0.0,0.0,MARG
3614,27AACCH2305Q1ZG,HEALTH N U THERAPEUTICS PVT LTD,2302072,2022-11-30,6053.20,726.38,0.0,0.0,MARG


In [35]:
# Function to create a detailed DataFrame for comparison
def create_comparison_df(matches_list, file_1_prefix="MARG_", file_2_prefix="PORTAL_"):
    comparison_data = []
    for match in matches_list:
        row_file_1, row_file_2 = match
        combined_row = {file_1_prefix + key: value for key, value in row_file_1.items()}
        combined_row.update({file_2_prefix + key: value for key, value in row_file_2.items()})
        comparison_data.append(combined_row)
    return pd.DataFrame(comparison_data)

# Creating detailed comparison DataFrames for each category
exact_matches_comparison_df = create_comparison_df(exact_matches)
under_5_percent_diff_comparison_df = create_comparison_df(under_5_percent_diff)
under_10_percent_diff_comparison_df = create_comparison_df(under_10_percent_diff)
more_than_10_percent_diff_comparison_df = create_comparison_df(more_than_10_percent_diff)

# Previewing the first few rows of the "Exact Matches" comparison DataFrame as an example
exact_matches_comparison_df.head()


,MARG_GST,MARG_Trade Name,MARG_Doc Number,MARG_Date,MARG_Taxable Value,MARG_IGST,MARG_CGST,MARG_SGST,MARG_REM,PORTAL_GST,PORTAL_Trade Name,PORTAL_Doc Number,PORTAL_Date,PORTAL_Taxable Value,PORTAL_IGST,PORTAL_CGST,PORTAL_SGST,PORTAL_REM
0,03AAECJ9456C1ZP,JOLLY HEALTHCARE PVT LTD OCT 2020,519,2022-04-29,3996.48,199.82,0.0,0.0,MARG,03AAECJ9456C1ZP,JOLLY HEALTH CARE PRIVATE LIMITED,JHC/519/2022-23,2022-04-29,3996.48,199.82,0.0,0.0,PORTAL
1,03AAECJ9456C1ZP,JOLLY HEALTHCARE PVT LTD OCT 2020,593,2022-08-25,1845.69,92.28,0.0,0.0,MARG,03AAECJ9456C1ZP,JOLLY HEALTH CARE PRIVATE LIMITED,JHC/593/2022-23,2022-08-25,1845.69,92.28,0.0,0.0,PORTAL
2,03AAECJ9456C1ZP,JOLLY HEALTHCARE PVT LTD OCT 2020,604,2022-09-09,164401.92,19728.23,0.0,0.0,MARG,03AAECJ9456C1ZP,JOLLY HEALTH CARE PRIVATE LIMITED,JHC/604/2022-23,2022-09-09,164401.92,19728.23,0.0,0.0,PORTAL
3,03AAECJ9456C1ZP,JOLLY HEALTHCARE PVT LTD OCT 2020,609,2022-09-14,164401.92,19728.23,0.0,0.0,MARG,03AAECJ9456C1ZP,JOLLY HEALTH CARE PRIVATE LIMITED,JHC/609/2022-23,2022-09-14,164401.92,19728.23,0.0,0.0,PORTAL
4,03AAECJ9456C1ZP,JOLLY HEALTHCARE PVT LTD OCT 2020,634,2022-10-21,1845.69,92.28,0.0,0.0,MARG,03AAECJ9456C1ZP,JOLLY HEALTH CARE PRIVATE LIMITED,JHC/634/2022-23,2022-10-21,1845.69,92.28,0.0,0.0,PORTAL


In [36]:
more_than_10_percent_diff_comparison_df.to_csv('more10.csv')

In [37]:
exact_matches_comparison_df.to_csv('exact.csv')

In [38]:
under_10_percent_diff_comparison_df.to_csv('under10.csv')

In [39]:
under_5_percent_diff_comparison_df.to_csv('under5.csv')

In [40]:
not_found_in_marg_df.to_csv('not_in_marg.csv')

In [41]:
not_found_in_portal_df.to_csv('not_in_portal.csv')